Image Compression framework :

[End to end image compression framework ](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7999241)

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

import os

import torch
import torchvision
from torch import nn

In [11]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=img_transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=img_transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=4,shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
# code block to text image loader
print(trainloader)

In [0]:
CHANNELS = 3
HEIGHT = 32
WIDTH = 32

### Define the network

In [0]:
class Interpolate(nn.Module):
    def __init__(self, size, mode):
        super(Interpolate, self).__init__()
        self.interp = nn.functional.interpolate
        self.size = size
        self.mode = mode
        
    def forward(self, x):
        x = self.interp(x, size=self.size, mode=self.mode, align_corners=False)
        return x

In [1]:
class autoencoder(nn.Module):
    def __init__(self):
    super(autoencoder, self).__init__()
    self.encoder = nn.Sequential(
        nn.Conv2d(CHANNELS, 64, 3),
        nn.ReLU(True),
        nn.Conv2d(64, 64, 3, stride=2, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(True),
        nn.Conv2d(64, CHANNELS, 3)
    )
    self.decoder = nn.Sequential(
        Interpolate(size=HEIGHT, mode='bilinear')
        nn.ConvTranspose2d(CHANNELS, 64, 3, stride=2),
    )

ModuleNotFoundError: ignored

In [0]:
Reference : https://github.com/L1aoXingyu/pytorch-beginner/tree/master/08-AutoEncoder